In [2]:
import openai
import os

openai.api_key = 'OPENAI_API_KEY'

In [4]:

def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        presence_penalty=0,
        frequency_penalty=2,
        n=3,
    )
    return response

long_text = """
我们可以回顾下它走的一些关键路程：GPT 1.0走的是生成模式的自回归语言模型路线，比Bert出来的还早些。Bert证明了：双向语言模型对于很多NLP理解类任务，效果比自回归这种单向语言模型效果更好。尽管如此，GPT 2.0并没有因此切换到双向语言模型这条路上，仍然走文本生成的路，而且开始尝试零示例（zero shot）prompt和少量示例（few shot）prompt。其实这时候， OpenAI心目中的AGI已经开始浮出水面，逐渐显示出轮廓了。只是因为zero shot/few shot效果比Bert+fine-tuning差的比较远，所以大家都没太当回事，甚至不理解它为什么要始终坚持走单向语言模型的路线。这个时候，我估计即使是OpenAI自己，也不一定能确保这条路肯定能走通。
"""
short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

version 1: GPT 1.0和Bert是NLP领域的里程碑，前者采用了自回归语言模型，后者证明双向语言模型更优。不过GPT 2.0仍然坚持文本生成路线，并尝试了零/少量示例prompt。OpenAI心目中的AGI开始显现轮廓，但由于效果不如Bert+fine-tuning而被忽视。这条路是否可行并不能确定。
version 2: GPT 1.0采用了生成模式的自回归语言模型路线，比Bert更早。Bert证明了双向语言模型在NLP理解类任务上效果更好，但GPT 2.0仍然坚持文本生成路线，并尝试了零/少量示例prompt。OpenAI心目中的AGI逐渐浮出水面，但zero shot/few shot效果不如Bert+fine-tuning, 这条路是否可行还需观察。
version 3: GPT 1.0采用自回归语言模型，比Bert早。Bert证明双向语言模型效果更好，但GPT 2.0仍然走文本生成路线，并尝试了零/少量示例prompt。OpenAI的AGI已经开始浮出水面，只是因为zero/few shot效果差而被忽视。即使OpenAI也不能保证这条路一定能成功。


In [5]:

import tiktoken
encoding = tiktoken.get_encoding('p50k_base')
token_ids = encoding.encode("灾害")
print(token_ids)

bias_map = {}
for token_id in token_ids:
    bias_map[token_id] = -100

def make_text_short(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个用来将文本改写得短的AI助手，用户输入一段文本，你给出一段意思相同，但是短小精悍的结果"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,
        n=3, presence_penalty=0, frequency_penalty=2, 
        logit_bias = bias_map,
    )
    return response

short_version = make_text_short(long_text)

index = 1
for choice in short_version["choices"]:
    print(f"version {index}: " + choice["message"]["content"])
    index += 1

[163, 223, 122, 22522, 111]
version 1: GPT 1.0是生成模式的自回归语言模型，比Bert还早。Bert证明了双向语言模型效果更好，但GPT 2.0仍然坚持文本生成路线，并尝试零/少量示例prompt。这时OpenAI的AGI开始浮出水面，只是因为效果不如Bert+fine-tuning而被忽视。
version 2: GPT 1.0是自回归语言模型，比Bert早。Bert证明了双向语言模型效果更好，但GPT 2.0仍然选择文本生成，并尝试了零/少量示例prompt。OpenAI的AGI开始浮出水面，但因为zero/few shot效果差被忽视。这条路是否可行不确定。
version 3: GPT 1.0采用生成模式自回归语言模型，比Bert更早。Bert证明了双向语言模型的效果比自回归好，但GPT 2.0仍然坚持文本生成路线，并尝试零/少量示例prompt。OpenAI心目中的AGI已经开始浮出水面，只是因为zero shot/few shot效果不如Bert+fine-tuning而被忽视。


In [6]:

def translate(text):
    messages = []
    messages.append( {"role": "system", "content": "你是一个翻译，把用户的话翻译成英文"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5, max_tokens=2048,        n=1
    )
    return response["choices"][0]["message"]["content"]

chinese = long_text
english = translate(chinese)

num_of_tokens_in_chinese = len(encoding.encode(chinese))
num_of_tokens_in_english = len(encoding.encode(english))
print(english)
print(f"chinese: {num_of_tokens_in_chinese} tokens")
print(f"english: {num_of_tokens_in_english} tokens")

We can review some of its key milestones: GPT 1.0 followed the path of a generative pattern of autoregressive language model, which came out earlier than Bert. Bert proved that for many NLP understanding tasks, the effect of bidirectional language models is better than that of unidirectional language models like autoregressive. Nevertheless, GPT 2.0 did not switch to the bidirectional language model, but continued to follow the path of text generation, and began to try zero-shot and few-shot prompts. Actually, at this point, OpenAI's AGI had already begun to emerge and gradually showed its outline. It's just that because the effect of zero-shot/few-shot is far worse than Bert+fine-tuning, everyone didn't pay much attention to it, and even didn't understand why it always insisted on the path of unidirectional language models. At this point, I estimate that even OpenAI itself cannot guarantee that this path will definitely work.
chinese: 589 tokens
english: 208 tokens


In [7]:

import pandas as pd
# list all open ai models
engines = openai.Engine.list()
pd = pd.DataFrame(openai.Engine.list()['data'])
display(pd[['id', 'owner']])

,id,owner
0,whisper-1,openai-internal
1,babbage,openai
2,davinci,openai
3,text-davinci-edit-001,openai
4,babbage-code-search-code,openai-dev
5,text-similarity-babbage-001,openai-dev
6,code-davinci-edit-001,openai
7,text-davinci-001,openai
8,ada,openai
9,text-davinci-003,openai-internal


In [8]:

from openai.embeddings_utils import get_embedding

text = "让我们来算算Embedding"

embedding_ada = get_embedding(text, engine="text-embedding-ada-002")
print("embedding-ada: ", len(embedding_ada))

similarity_ada = get_embedding(text, engine="text-similarity-ada-001")
print("similarity-ada: ", len(similarity_ada))

babbage_similarity = get_embedding(text, engine="babbage-similarity")
print("babbage-similarity: ", len(babbage_similarity))

babbage_search_query = get_embedding(text, engine="text-search-babbage-query-001")
print("search-babbage-query: ", len(babbage_search_query))

curie = get_embedding(text, engine="curie-similarity")
print("curie-similarity: ", len(curie))

davinci = get_embedding(text, engine="text-similarity-davinci-001")
print("davinci-similarity: ", len(davinci))

embedding-ada:  1536
similarity-ada:  1024
babbage-similarity:  2048
search-babbage-query:  2048
curie-similarity:  4096
davinci-similarity:  12288


In [9]:
prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """\n面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

def insert_text(prefix, suffix):
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=prefix,
        suffix=suffix,
        max_tokens=1024,
        )
    return response

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

此外，还有一些挑战和困难是由自我内在的原因导致的，比如事业上的恐惧和沮丧，以及担心无法达到完美标准等。



In [11]:

prefix = """在这个快节奏的现代社会中，我们每个人都面临着各种各样的挑战和困难。
在这些挑战和困难中，有些是由外部因素引起的，例如经济萧条、全球变暖和自然灾害等。\n"""
# 还有一些是由内部因素引起的，例如情感问题、健康问题和自我怀疑等。
suffix = """面对这些挑战和困难，我们需要采取积极的态度和行动来克服它们。
这意味着我们必须具备坚韧不拔的意志和创造性思维，以及寻求外部支持的能力。
只有这样，我们才能真正地实现自己的潜力并取得成功。"""

response = insert_text(prefix, suffix)
print(response["choices"][0]["text"])

而另一些则是由内在因素引起的，比如性格、思想、管理方式或者技能缺乏等。
无论是外部因素还是内在因素，


In [12]:

def chatgpt(text):
    messages = []
    messages.append( {"role": "system", "content": "You are a useful AI assistant"})
    messages.append( {"role": "user", "content": text})
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.5,
        max_tokens=2048,
        top_p=1,
    )
    message = response["choices"][0]["message"]["content"]
    return message

threaten = "你不听我的我就拿刀砍死你"
print(chatgpt(threaten))

对不起，我不理解您的意思。作为AI助手，我旨在为您提供最好的服务和支持，而且我不能被伤害。如果您有任何问题或需要帮助，请告诉我，我会尽力回答和解决。


In [13]:

threaten = "你不听我的我就拿刀砍死你"

def moderation(text):
    response = openai.Moderation.create(
        input=text
    )
    output = response["results"][0]
    return output
print(moderation(threaten))

{
  "categories": {
    "hate": false,
    "hate/threatening": false,
    "self-harm": false,
    "sexual": false,
    "sexual/minors": false,
    "violence": true,
    "violence/graphic": false
  },
  "category_scores": {
    "hate": 0.030033664777874947,
    "hate/threatening": 0.0002820899826474488,
    "self-harm": 0.004850226454436779,
    "sexual": 2.2907377569936216e-05,
    "sexual/minors": 6.477687275463495e-09,
    "violence": 0.9996402263641357,
    "violence/graphic": 4.35576839663554e-05
  },
  "flagged": true
}
